## Functions in PyBEAM's defualt sub-module

In this notebook we detail all the functions and features available to the PyBEAM default module. This describes their usage and inputs, in addition to examples of how to use the functions.


In [ ]:
"""
How to import PyBEAM's default sub-module.

"""

import pybeam.default as pbd


In [ ]:
"""
model (dict): How to build a model.
How to build a model using PyBEAM's default sub-module.

"""

# if model type is 'base', these six keys are required

model = {'type' : 'base',  # model type ('base' or 'ugm')
        'sigma' : 1.0,     # sets sigma, the scaling parameter
    'threshold' : 'fixed', # sets threshold type (fixed, linear, exponential, or weibull)
      'leakage' : False,   # if True, drift rate has leaky integration
        'delay' : False,   # if True, decision threshold motion is delayed (only for non-fixed thresholds)
'contamination' : False}   # if True, uniform contamination added to model

# if model type is 'ugm', these three keys are required

# model = {'type' : 'ugm',  # model type ('base' or 'ugm')
#         'sigma' : 1.0,    # sets sigma, the scaling parameter
# 'contamination' : False}  # if True, uniform contamination added to model


In [ ]:
"""
function: parse_model
Takes input of model dictionary and returns what parameters are used in that model.

Inputs:

    model (dict) -
        Dictionary containing model information. Must be 'base' or 'ugm' (see paper for description of
        these models). See cell above.
        
Outputs:

    (list) containing all parameters used in model. These are also the keys used in the phi and conditions
    dictionaries used in other functions (see below).

"""

# in this case, it will output the priors used for only the base model, no extenions. These are:
#     t_nd: the non-decision time
#       w : the relative start point
#      mu : thre drift rate
#       a : decision threshold location

pbd.parse_model(model)


In [ ]:
"""
function: simulate_model
Simulates model defined in model dictionary.

Inputs:

    N_sims (int) -
        Number of accumulators to simulate.

    model (dict) -
        Dictionary containing model information. See model description.

    phi (dict) -
        Dictionary containing model parameter values. Keys are the model parameters, while values are the 
        value associated with that parameter. If keys are unknown for your model, run function parse_model.
        The list output by that function provides the keys for this dictionary.

    seed (int, False); OPTIONAL, defaults to False -
        Sets the random number seed. Defaults to False. If False, program randomly chooses a seed.

    dt (float) OPTIONAL - 
        Sets the simulation time step. By default, dt = 1.0e-4.
        
Outputs:

    (dict) containing two keys: 'rt_upper' and 'rt_lower'. These contain the reaction times for the upper
    and lower threshold crossings, respectively.

"""

# keys for this function must be those output by parse_model
phi = {'t_nd' : 0.25,
          'w' : 0.5,
         'mu' : 1.0,
          'a' : 0.5}

rt = pbd.simulate_model(N_sims = 500,   
                         model = model,
                           phi = phi,
                          seed = 1,
                            dt = 1.0e-4)

rt


In [ ]:
"""
function: model_rt
Calculate the model's predicted rt distribution (model likelihood function).

Inputs:

    model (dict) -
        Dictionary containing model information. See model description.

    phi (dict) -
        Dictionary containing model parameter values. Keys are the model parameters, while values are the 
        value associated with that parameter. If keys are unknown for your model, run function parse_model.
        The list output by that function provides the keys for this dictionary.

    x_res (str), OPTIONAL - 
        Sets spatial resolution of Fokker-Planck solver (Crank-Nicolson algorithm). 'default', the default 
        setting, places 101  spatial steps between the two decision thresholds. 'high' places 151 spatial 
        steps, 'very_high' places 251 spatial steps, while 'max' places 501 spatial steps. 'default' is
        recommened in nearly all cases.
        
    t_res (str), OPTIONAL - 
        Sets temporal resolution of Fokker-Planck solver (Crank-Nicolson algorithm). The time step used
        by the solver is set by simulating 10 accumulators from the model and finding their average rt.
        This is then mulitiplied by a constant determined by t_res. 'default', the default setting,
        multiplies this by 0.025. 'high' multiplies this by 0.0175, 'very_high' by 0.01, and 'max' by
        0.005. 'default' is recommended in nearly all cases.
        
Outputs:

    (dict) containing three keys: 'time',  rt_upper' and 'rt_lower'. These contain the reaction time 
    distributions (likelihood functions) for the upper and lower threshold crossings, respectively.

"""

# keys for this function must be those output by parse_model
phi = {'t_nd' : 0.25,
          'w' : 0.5,
         'mu' : 1.0,
          'a' : 0.5}

rt = pbd.simulate_model(N_sims = 500,   
                         model = model,
                           phi = phi,
                          seed = 1,
                            dt = 1.0e-4)

pbd.model_rt(model = model, 
               phi = phi, 
             x_res = 'default', 
             t_res = 'default')


In [ ]:
"""
function: model_loglike
Calculate the loglikelihood of a data set from the model's likelihood function.

Inputs:

    model (dict) -
        Dictionary containing model information. See model description.

    phi (dict) -
        Dictionary containing model parameter values. Keys are the model parameters, while values are the 
        value associated with that parameter. If keys are unknown for your model, run function parse_model.
        The list output by that function provides the keys for this dictionary.
        
    rt (dict) -
        Dictionary contining two keys 'rt_upper' and 'rt_lower'. The former contians the reaction time data
        for the upper threshold crossing, while the latter contains the data for the lower decison threshold
        crossing.

    x_res (str), OPTIONAL - 
        Sets spatial resolution of Fokker-Planck solver (Crank-Nicolson algorithm). 'default', the default 
        setting, places 101  spatial steps between the two decision thresholds. 'high' places 151 spatial 
        steps, 'very_high' places 251 spatial steps, while 'max' places 501 spatial steps. 'default' is
        recommened in nearly all cases.
        
    t_res (str), OPTIONAL - 
        Sets temporal resolution of Fokker-Planck solver (Crank-Nicolson algorithm). The time step used
        by the solver is set by simulating 10 accumulators from the model and finding their average rt.
        This is then mulitiplied by a constant determined by t_res. 'default', the default setting,
        multiplies this by 0.025. 'high' multiplies this by 0.0175, 'very_high' by 0.01, and 'max' by
        0.005. 'default' is recommended in nearly all cases.
        
Outputs:

    (float) The loglikelihood of the input data set calculated from the model rt (likelihood function).

"""

# keys for this function must be those output by parse_model
phi = {'t_nd' : 0.25,
          'w' : 0.5,
         'mu' : 1.0,
          'a' : 0.5}

rt = pbd.simulate_model(N_sims = 500,   
                         model = model,
                           phi = phi,
                          seed = 1,
                            dt = 1.0e-4)

pbd.model_loglike(model = model, 
                    phi = phi,
                     rt = rt,
                  x_res = 'default', 
                  t_res = 'default')


In [ ]:
"""
function: plot_rt
    Generates a figure containing the model rt distributions (red lines) and, optionally is rt data is given,
    the data rt distribution (grey histogram).

Inputs:

    model (dict) -
        Dictionary containing model information. See model description.

    phi (dict) -
        Dictionary containing model parameter values. Keys are the model parameters, while values are the 
        value associated with that parameter. If keys are unknown for your model, run function parse_model.
        The list output by that function provides the keys for this dictionary.
        
    rt (False, dict) OPTIONAL -
        Dictionary contining two keys 'rt_upper' and 'rt_lower'. The former contians the reaction time data
        for the upper threshold crossing, while the latter contains the data for the lower decison threshold
        crossing.

    x_res (str), OPTIONAL - 
        Sets spatial resolution of Fokker-Planck solver (Crank-Nicolson algorithm). 'default', the default 
        setting, places 101  spatial steps between the two decision thresholds. 'high' places 151 spatial 
        steps, 'very_high' places 251 spatial steps, while 'max' places 501 spatial steps. 'default' is
        recommened in nearly all cases.
        
    t_res (str), OPTIONAL - 
        Sets temporal resolution of Fokker-Planck solver (Crank-Nicolson algorithm). The time step used
        by the solver is set by simulating 10 accumulators from the model and finding their average rt.
        This is then mulitiplied by a constant determined by t_res. 'default', the default setting,
        multiplies this by 0.025. 'high' multiplies this by 0.0175, 'very_high' by 0.01, and 'max' by
        0.005. 'default' is recommended in nearly all cases.
        
    bins (False, int), OPTIONAL -
        Sets how many histogram bins to be used to plot the data. If False, automatically sets the amount
        of bins using the Freedman Diaconis Estimator.
        
Outputs:

    (fig) Figure containing the model rt distribution (red) and the rt data (optional, grey histogram).

"""

# keys for this function must be those output by parse_model
phi = {'t_nd' : 0.25,
          'w' : 0.5,
         'mu' : 1.0,
          'a' : 0.5}

rt = pbd.simulate_model(N_sims = 500,   
                         model = model,
                           phi = phi,
                          seed = 1,
                            dt = 1.0e-4)

pbd.plot_rt(model = model, 
              phi = phi, 
            x_res = 'default', 
            t_res = 'default', 
               rt = rt, 
             bins = 25)


In [ ]:
"""
function: inference
    Runs MCMC routine using PyMC3.

Inputs:

    model (dict) -
        Dictionary containing model information. See model description.

    priors (dict) -
        Dictionary containing parameter priors. Key names are arbitrary. The values for each key are 
        strings written in PyMC3's syntax for priors. They can also be constants if you want a parameter
        to remain fixed at all times. See below for example.
        
    conditions (dict) -
        Dictionary containing dictionaries for each model condition. See below and example files for use.
        
    samples (int) -
        Sets the number of MCMC samples to run. Recommend at least 25000 samples.
        
    chains (int) -
        Sets the number of MCMC chains to run. Recomend at least 3 chains.
        
    cores (int) - 
        Sets the number of cpu cores to run the chains on.
        
    file_name (str) -
        Sets the name of the .nc file output by the solver containing the posterios. Automatically adds 
        the .nc extension to the string.
        
    solver (str), OPTIONAL -
        Sets the MCMC algorithm used by PyMC3. Defaults to 'DEMetropolisZ' (recommended), but can be changed
        to 'DEMetropolis' and 'Slice'.

    x_res (str), OPTIONAL - 
        Sets spatial resolution of Fokker-Planck solver (Crank-Nicolson algorithm). 'default', the default 
        setting, places 101  spatial steps between the two decision thresholds. 'high' places 151 spatial 
        steps, 'very_high' places 251 spatial steps, while 'max' places 501 spatial steps. 'default' is
        recommened in nearly all cases.
        
    t_res (str), OPTIONAL - 
        Sets temporal resolution of Fokker-Planck solver (Crank-Nicolson algorithm). The time step used
        by the solver is set by simulating 10 accumulators from the model and finding their average rt.
        This is then mulitiplied by a constant determined by t_res. 'default', the default setting,
        multiplies this by 0.025. 'high' multiplies this by 0.0175, 'very_high' by 0.01, and 'max' by
        0.005. 'default' is recommended in nearly all cases.
        
    tune (int), OPTIONAL -
        Sets the amount of MCMC tuning steps. Defaults to zero (recommended for DEMetropolisZ and
        DEMetropolis, but can sometimes be useful. Test on data if need be). If using Slice, 
        tuning steps are necessary (recommend at least 500).
        
Outputs:

    (trace object) Posterior trace information.

    (file) Outputs .nc file containing all trace information. See pycm3/arviz documentation for how
    to work with this file type.

"""

# dictionary containing priors. Keys will be referenced by the conditions diciontaries, while the values
# are distributions written in PyMC3 syntax. In this case, we use the syntax for Uniform priors from
# PyMC3's docs.

p = {'pt_nd' : 'Uniform("t_nd", lower = 0.0, upper = 0.75)', # prior for t_nd
        'pw' : 'Uniform("w", lower = 0.3, upper = 0.7)', # prior for w
       'pmu' : 'Uniform("mu", lower = -5.0, upper = 5.0)', # prior for mu
        'pa' : 'Uniform("a", lower = 0.25, upper = 1.5)'} # prior for a

# if you wanat a parameter to remain constant, input its constant value in this dictionary
# p = {'pt_nd' : 'Uniform("t_nd", lower = 0.0, upper = 0.75)', # prior for t_nd
#         'pw' : 0.5, # prior for w
#        'pmu' : 'Uniform("mu", lower = -5.0, upper = 5.0)', # prior for mu
#         'pa' : 'Uniform("a", lower = 0.25, upper = 1.5)'} # prior for a

# the condition dicionary contains sub-dictionaries contianing each conditions model information. We
# first define the sub-condtion. It contains the keys for the model parameters output by the parse model
# function (in this case t_nd, w, mu, and a). It also must contain a key, 'rt' referencing the data set.
# The value of this key must be a dictionary of the form output by the simulate model function.
# The value for each other key references the relevant prior from the prior dictionary.

c = {'rt' : rt,
   't_nd' : 'pt_nd',
      'w' : 'pw',
     'mu' : 'pmu',
      'a' : 'pa'}

# we load this condition into a last dictionary which gets input into the function. This dictionary has keys
# ranging from 0, 1, ... , number of conditions - 1. In this case, we only have one condition, so there is
# only one key.

cond = {0 : c}

# we now call the function

pbd.inference(model = model,
             priors = p, 
         conditions = cond,
            samples = 25000, 
             chains = 3,
              cores = 3,
          file_name = 'description',
             solver = 'DEMetropolisZ', 
              x_res = 'default', 
              t_res = 'default',
               tune = 0)


In [ ]:
"""
function: plot_trace
    Plots figure containing all posteriors.

Inputs:

    file_name (str) -
        File name as a string. Must be a .nc file. Do not include the .nc extension in the string,
        it is automatically added by the program.
        
    burnin (int) -
        Number of samples to throw out form the posterior when plotting.
        
Outputs:

    (fig) Figure containing all posteriors.

"""

pbd.plot_trace(file_name = 'description', burnin = 12500);


In [ ]:
"""
function: summary
    Generates data frame containing useful posetior information. Pulled from the arviz function, see their
    documentation for understaning table information.

Inputs:

    file_name (str) -
        File name as a string. Must be a .nc file. Do not include the .nc extension in the string,
        it is automatically added by the program.
        
    burnin (int) -
        Number of samples to throw out form the posterior when plotting.
        
Outputs:

    (df) Data frame containing useful trace information.

"""

pbd.summary(file_name = 'description', burnin = 12500)
